In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()
model = ChatOpenAI(model='gpt-3.5-turbo')

In [2]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store =  {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model , get_session_history) # wraps model and adds message history function which takes in session id to distinguis seperate sessions

config = {"configurable":{"session_id" : "abc2"}} # configurable which is passed into runnable every time

response = with_message_history.invoke(
    [HumanMessage(content="hi , I am Roy")],
    config= config
)

response.content

Parent run 60f3c166-768b-4f5d-b071-f1f884cb71ee not found for run 2efd82ae-eb1d-4ddb-82f6-897af4df2b98. Treating as a root run.


'Hello Roy! How can I assist you today?'

In [3]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config= config
)

response.content

Parent run 4a963092-eb2b-4934-b233-243c559e08f0 not found for run de9ace4d-8143-4050-8daa-ab3f959b74a7. Treating as a root run.


'Your name is Roy.'

In [5]:
# different session id
config = {"configurable":{"session_id": "abc3"}}
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config= config
)

response.content

Parent run bc5b32a9-3aa1-4366-ab86-25fed8bc78fe not found for run 48d8a9f8-cd24-428a-8705-4dc26a2cd376. Treating as a root run.


"I'm sorry, I do not have access to personal information such as your name."

### Prompt Template

In [7]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [("system" , "You are a helpful assistant. Answer all questions to the best of your ability."), 
     MessagesPlaceholder(variable_name="messages")]
)

chain = prompt | model

response = chain.invoke({"messages" : [HumanMessage(content="Hi, I am Roy")]})
response.content

'Hello, Roy! How can I assist you today?'

In [8]:
with_message_history = RunnableWithMessageHistory(chain , get_session_history)
config = {"configurable":{"session_id": "abc4"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi , I am Roy")],
    config= config
)

response.content

Parent run 43256ef6-78e8-43be-8fa1-caeab716e310 not found for run ef031855-c0fa-4322-be51-1cf4a10766e4. Treating as a root run.


'Hello Roy! How can I assist you today?'

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 95cdc0f6-42cc-4bfc-a936-e6d425a90330 not found for run 063e1b38-c5eb-42bb-b9f2-1700d88586d2. Treating as a root run.


'Your name is Roy. How can I assist you further, Roy?'

In [10]:
# more complicated prompt

prompt = ChatPromptTemplate.from_messages([
    ("system" , "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm Roy")], "language": "Spanish"})
response.content

'¡Hola Roy! ¿En qué puedo ayudarte hoy?'

In [11]:
with_message_history = RunnableWithMessageHistory( chain ,  get_session_history , input_messages_key= "messages")
config = {"configurable":{"session_id":"abc5"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm Roy")], "language": "Spanish"}, config
)
response.content

Parent run f5d93047-bd8e-4e2e-b320-256d41db8c3e not found for run 1f51ac23-8f16-44b2-a4c5-754794d088ed. Treating as a root run.


'¡Hola, Roy! ¿En qué puedo ayudarte hoy?'

In [12]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="what's my name")] , "language": "Spanish"}, config
)
response.content

Parent run 2f45b0f9-7d9f-499f-a7fe-e2810fa106b2 not found for run b610f7a4-db5c-4166-bf83-c51f8831cdde. Treating as a root run.


'Tu nombre es Roy.'

In [ ]:
# we need to limit the size of the context window. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. 

In [13]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [14]:
from langchain_core.messages import AIMessage

messages = [
    HumanMessage(content="hi! I'm Roy"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [15]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to personal information about you."

In [16]:
# if we ask icecrema questoin
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'Your favorite ice cream flavor is vanilla.'

In [17]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run b3441daf-19d4-470b-8916-b1cf16241767 not found for run 39d98ec7-6566-41ba-9c92-89f7eefcfd87. Treating as a root run.


"I'm sorry, I don't have access to that information."

In [18]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run 2b268b41-9e16-4829-bf2b-a0eec2557586 not found for run b0cff5b1-22f9-4c9a-b094-de351fbbc83a. Treating as a root run.


"I'm sorry, I don't have that information."

In [ ]:
# two new messages in the chat history , the info needed in fone

## Streaming

In [22]:
# as some times chatbot take time to respond , stremaing the reponse would return intermediate response

In [23]:
config = {"configurable":{"session_id":"abc30"}}
for r in with_message_history.stream(
        {"messages":[HumanMessage(content="Tell me a joke")] , "language":"English"}, config= config
    ):
    print(r.content , end = "|")

Parent run a5fd7885-bdd4-4eac-b9c2-78c6a196f009 not found for run aad86902-3032-4a47-95c1-0c1ae242ad05. Treating as a root run.


|Sure|,| here|'s| a| joke| for| you|:| Why| couldn|'t| the| bicycle| stand| up| by| itself|?| Because| it| was| two|-t|ired|!||